In [1]:
## Dependencies

import pandas as pd
import pathlib as path



import requests
import json
from pprint import pprint

# import numpy as np
# from scipy.stats import linregress

# import matplotlib.pyplot as plt
# import hvplot.pandas
# import geopandas as gpd

# (1) collect immigration data 

##### Data source from IRCC web site
https://www.ircc.canada.ca/opendata-donneesouvertes/data/ODP-PR-Citz.csv
from
https://ouvert.canada.ca/data/fr/dataset/f7e5498e-0ad8-4417-85c9-9b8aff9b9eda/resource/5f5fa9ca-b621-4dac-91d1-25654a25735c

In [2]:
### import and read csv file

path_imm = 'https://www.ircc.canada.ca/opendata-donneesouvertes/data/ODP-PR-Citz.csv'
imm_df = pd.read_csv(path_imm, delimiter='\t')

imm_df.to_csv('Output/immigration_raw_data.csv')
imm_df.head(5)

,EN_YEAR,EN_QUARTER,EN_MONTH,FR_ANNEÉ,FR_TRIMESTRE,FR_MOIS,EN_COUNTRY_OF_CITIZENSHIP,FR_PAYS_DE_CITOYENNETÉ,TOTAL
0,2015,Q1,Feb,2015,T1,fév.,Afghanistan,Afghanistan,125
1,2015,Q1,Feb,2015,T1,fév.,Albania,Albanie,25
2,2015,Q1,Feb,2015,T1,fév.,Algeria,Algérie,125
3,2015,Q1,Feb,2015,T1,fév.,Antigua and Barbuda,Antigua-et-Barbuda,5
4,2015,Q1,Feb,2015,T1,fév.,Argentina,Argentine,15


In [3]:
#check datatype 

imm_df.dtypes

EN_YEAR                       int64
EN_QUARTER                   object
EN_MONTH                     object
FR_ANNEÉ                      int64
FR_TRIMESTRE                 object
FR_MOIS                      object
EN_COUNTRY_OF_CITIZENSHIP    object
FR_PAYS_DE_CITOYENNETÉ       object
TOTAL                        object
dtype: object

In [4]:
# drop useless columns

imm_clean_df = imm_df.drop(columns=['FR_ANNEÉ','FR_TRIMESTRE','FR_MOIS','FR_PAYS_DE_CITOYENNETÉ'])
imm_clean_df.columns

Index(['EN_YEAR', 'EN_QUARTER', 'EN_MONTH', 'EN_COUNTRY_OF_CITIZENSHIP',
       'TOTAL'],
      dtype='object')

In [7]:
# check empty values with 1 example raw 42

print(imm_clean_df.loc[42,])

EN_YEAR                          2015
EN_QUARTER                         Q1
EN_MONTH                          Feb
EN_COUNTRY_OF_CITIZENSHIP    Dominica
TOTAL                              --
Name: 42, dtype: object


In [8]:
# convert Total to numeric

imm_clean_df['TOTAL'] = pd.to_numeric(imm_clean_df['TOTAL'], errors='coerce')

# replace empty with 0 

imm_clean_df = imm_clean_df.fillna({'TOTAL': 0})
print(imm_clean_df.loc[42,])

EN_YEAR                          2015
EN_QUARTER                         Q1
EN_MONTH                          Feb
EN_COUNTRY_OF_CITIZENSHIP    Dominica
TOTAL                             0.0
Name: 42, dtype: object


In [9]:
# check that total is numeric _ integer

imm_clean_df.dtypes

EN_YEAR                        int64
EN_QUARTER                    object
EN_MONTH                      object
EN_COUNTRY_OF_CITIZENSHIP     object
TOTAL                        float64
dtype: object

In [ ]:
# To DELETE ???

# imm_clean_df = imm_clean_df.drop(columns=['FR_ANNEÉ','FR_TRIMESTRE','FR_MOIS','FR_PAYS_DE_CITOYENNETÉ'])
# imm_clean_df.columns

In [11]:
# rename columns 

imm_clean_df = imm_clean_df.rename(columns={
                                            'EN_YEAR'            :           'year' ,
                                            'EN_QUARTER'         :          'quarter' , 
                                            'EN_MONTH'           :          'month_str' ,
                                            'EN_COUNTRY_OF_CITIZENSHIP' :   'Country' ,
                                            'TOTAL'              :           'total'
                                            })
imm_clean_df.dtypes

year           int64
quarter       object
month_str     object
Country       object
total        float64
dtype: object

In [12]:
# add months with numerical values
 ## check that there is no strange value 

months_str = imm_clean_df['month_str'].unique()
months_str

array(['Feb', 'Jan', 'Mar', 'Apr', 'Jun', 'May', 'Aug', 'Jul', 'Sep',
       'Dec', 'Nov', 'Oct'], dtype=object)

In [13]:
# use of Mapping to add integers to month abbreviations

month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Map month abbreviations to integers using the dictionary
imm_clean_df['month_int'] = imm_clean_df['month_str'].map(month_mapping)

print(imm_clean_df.head())

   year quarter month_str              Country  total  month_int
0  2015      Q1       Feb          Afghanistan  125.0          2
1  2015      Q1       Feb              Albania   25.0          2
2  2015      Q1       Feb              Algeria  125.0          2
3  2015      Q1       Feb  Antigua and Barbuda    5.0          2
4  2015      Q1       Feb            Argentina   15.0          2


In [15]:
# extract useless values from country name (after the comma)
## example Congo

print(imm_clean_df.loc[34,])

year                                    2015
quarter                                   Q1
month_str                                Feb
Country      Congo, People's Republic of the
total                                    0.0
month_int                                  2
Name: 34, dtype: object


In [16]:
# split the country name column using delimiter ','

imm_clean_df[['country', 'country_add']] = imm_clean_df['Country']\
                                                    .str.split(',', expand=True)

print(imm_clean_df.loc[34,])

year                                      2015
quarter                                     Q1
month_str                                  Feb
Country        Congo, People's Republic of the
total                                      0.0
month_int                                    2
country                                  Congo
country_add           People's Republic of the
Name: 34, dtype: object


In [17]:
# keep only country name cleaned 

imm_clean_df = imm_clean_df.drop(columns=['Country','country_add'])
imm_clean_df.columns

Index(['year', 'quarter', 'month_str', 'total', 'month_int', 'country'], dtype='object')

In [19]:
# check with congo

print(imm_clean_df.loc[34,])

year          2015
quarter         Q1
month_str      Feb
total          0.0
month_int        2
country      Congo
Name: 34, dtype: object


In [20]:
# groupby country / year to get the total by country

imm_yearly = imm_clean_df.groupby(['country','year'])['total'].sum()
imm_yearly.to_csv('Output/immigrants_by_country_year.csv')
imm_yearly

country      year
Afghanistan  2015    2625.0
             2016    2655.0
             2017    3460.0
             2018    3560.0
             2019    3890.0
                      ...  
Zimbabwe     2020     335.0
             2021     505.0
             2022     615.0
             2023     555.0
             2024     505.0
Name: total, Length: 1887, dtype: float64

In [21]:
# groupby to get the total by country

imm_bycountry = imm_clean_df.groupby(['country'])['total'].sum()
imm_bycountry.to_csv('Output/immigrants_by_country.csv')
imm_bycountry

country
Afghanistan       81305.0
Albania            5540.0
Algeria           39655.0
Andorra               0.0
Angola              925.0
                   ...   
Virgin Islands        0.0
Western Sahara        0.0
Yemen              5875.0
Zambia              605.0
Zimbabwe           4610.0
Name: total, Length: 213, dtype: float64

# (2) create DF for countries

#### information will be used to merge or generate vizualisations
source : https://api.worldbank.org/v2/country?format=json

In [ ]:
country_list = imm_clean_df['country'].tolist()
country_list

In [24]:
## retrieve all the countries from world bank API

### generate DF Countries

page = 1   ## there are 6 pages in the json file
data = []
url_countries = 'https://api.worldbank.org/v2/country?format=json'

for page in range(1,7):
    response = requests.get(f"{url_countries}&page={page}")
    json_data = response.json()
    
    # Add the data from the current page
    data.extend(json_data[1])  
    
## create DF and select columns

countries_wb_df = pd.DataFrame(data)
countries_wb_df['region'] = countries_wb_df['region'].apply(lambda x: x['value'])
countries_wb_df = countries_wb_df[['id', 'iso2Code', 'name', 'region','capitalCity', 'longitude', 'latitude']]
countries_wb_df = countries_wb_df.rename(columns={
                                                    'name':'country',
                                                    'id':'iso3Code'
                                                    })


regions = (countries_wb_df['region'] != "Aggregates")
countries_wb_df = countries_wb_df[regions]


countries_wb_df.head()

,iso3Code,iso2Code,country,region,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,Oranjestad,-70.0167,12.5167
2,AFG,AF,Afghanistan,South Asia,Kabul,69.1761,34.5228
5,AGO,AO,Angola,Sub-Saharan Africa,Luanda,13.242,-8.81155
6,ALB,AL,Albania,Europe & Central Asia,Tirane,19.8172,41.3317
7,AND,AD,Andorra,Europe & Central Asia,Andorra la Vella,1.5218,42.5075


In [26]:
### Export to csv file in output

countries_wb_df.to_csv('Output/countries_list_UN_referential.csv')


# (3) Extract macro economic data
#### from world bank API
source : https://api.worldbank.org/v2/country/DZA/indicator/NY.GDP.PCAP.KD?date=2015:2024&format=json

In [27]:
# the API uses the iso3Code code of the country (example PAK for pakistan, AFG for Afghanistan)
# we need to identify the needed countries from from our immigration_df and retrieve these iso3Code from countries_df

countries_request = pd.merge(imm_clean_df, countries_wb_df, left_on='country', right_on='country')
countries_request

,year,quarter,month_str,total,month_int,country,iso3Code,iso2Code,region,capitalCity,longitude,latitude
0,2015,Q1,Feb,125.0,2,Afghanistan,AFG,AF,South Asia,Kabul,69.1761,34.5228
1,2015,Q1,Feb,25.0,2,Albania,ALB,AL,Europe & Central Asia,Tirane,19.8172,41.3317
2,2015,Q1,Feb,125.0,2,Algeria,DZA,DZ,Middle East & North Africa,Algiers,3.05097,36.7397
3,2015,Q1,Feb,5.0,2,Antigua and Barbuda,ATG,AG,Latin America & Caribbean,Saint John's,-61.8456,17.1175
4,2015,Q1,Feb,15.0,2,Argentina,ARG,AR,Latin America & Caribbean,Buenos Aires,-58.4173,-34.6118
...,...,...,...,...,...,...,...,...,...,...,...,...
15932,2024,Q4,Oct,0.0,10,United Arab Emirates,ARE,AE,Middle East & North Africa,Abu Dhabi,54.3705,24.4764
15933,2024,Q4,Oct,5.0,10,Uruguay,URY,UY,Latin America & Caribbean,Montevideo,-56.0675,-34.8941
15934,2024,Q4,Oct,15.0,10,Uzbekistan,UZB,UZ,Europe & Central Asia,Tashkent,69.269,41.3052
15935,2024,Q4,Oct,0.0,10,Zambia,ZMB,ZM,Sub-Saharan Africa,Lusaka,28.2937,-15.3982


In [30]:
# extrac the needed iso3code in a list to be used to iterate

country_list = countries_request['iso3Code'].unique().tolist()
country_list

['AFG',
 'ALB',
 'DZA',
 'ATG',
 'ARG',
 'ARM',
 'AUS',
 'AUT',
 'AZE',
 'BHR',
 'BGD',
 'BRB',
 'BLR',
 'BEL',
 'BLZ',
 'BEN',
 'BTN',
 'BOL',
 'BWA',
 'BRA',
 'BGR',
 'BDI',
 'KHM',
 'CMR',
 'CAF',
 'TCD',
 'CHL',
 'CHN',
 'COL',
 'COM',
 'CRI',
 'HRV',
 'CUB',
 'CYP',
 'DNK',
 'DJI',
 'DMA',
 'DOM',
 'ECU',
 'SLV',
 'ERI',
 'ETH',
 'FJI',
 'FIN',
 'FRA',
 'GEO',
 'DEU',
 'GHA',
 'GRC',
 'GRD',
 'GTM',
 'GIN',
 'GUY',
 'HTI',
 'HND',
 'HUN',
 'ISL',
 'IND',
 'IDN',
 'IRQ',
 'IRL',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'XKX',
 'LVA',
 'LBN',
 'LBR',
 'LBY',
 'LTU',
 'MDG',
 'MYS',
 'MDV',
 'MLI',
 'MRT',
 'MUS',
 'MEX',
 'MDA',
 'MNG',
 'MNE',
 'MAR',
 'NAM',
 'NPL',
 'NLD',
 'NZL',
 'NIC',
 'NER',
 'NGA',
 'NOR',
 'PAK',
 'PAN',
 'PRY',
 'PER',
 'PHL',
 'POL',
 'PRT',
 'ROU',
 'RWA',
 'WSM',
 'SAU',
 'SEN',
 'SRB',
 'SLE',
 'SGP',
 'SVK',
 'SVN',
 'SOM',
 'ZAF',
 'ESP',
 'LKA',
 'LCA',
 'VCT',
 'SDN',
 'SWE',
 'CHE',
 'TJK',
 'TZA',
 'THA',
 'TGO',
 'TTO',
 'TUN',


In [22]:
import requests

In [31]:
# this line is just to check that we are connected to the APIT : O TEST before loop

link_pop = f'https://api.worldbank.org/v2/country/DZA/indicator/NY.GDP.PCAP.KD?date=2015:2024&format=json'
response_pop = requests.get(link_pop).json()

pprint(response_pop)

[{'lastupdated': '2024-12-16',
  'page': 1,
  'pages': 1,
  'per_page': 50,
  'sourceid': '2',
  'total': 9},
 [{'country': {'id': 'DZ', 'value': 'Algeria'},
   'countryiso3code': 'DZA',
   'date': '2023',
   'decimal': 1,
   'indicator': {'id': 'NY.GDP.PCAP.KD',
                 'value': 'GDP per capita (constant 2015 US$)'},
   'obs_status': '',
   'unit': '',
   'value': 4660.40545686886},
  {'country': {'id': 'DZ', 'value': 'Algeria'},
   'countryiso3code': 'DZA',
   'date': '2022',
   'decimal': 1,
   'indicator': {'id': 'NY.GDP.PCAP.KD',
                 'value': 'GDP per capita (constant 2015 US$)'},
   'obs_status': '',
   'unit': '',
   'value': 4544.46688059647},
  {'country': {'id': 'DZ', 'value': 'Algeria'},
   'countryiso3code': 'DZA',
   'date': '2021',
   'decimal': 1,
   'indicator': {'id': 'NY.GDP.PCAP.KD',
                 'value': 'GDP per capita (constant 2015 US$)'},
   'obs_status': '',
   'unit': '',
   'value': 4456.74687641147},
  {'country': {'id': 'DZ', 'valu

In [32]:
# this cell is used to check our code before to launch the loop 
# TO TEST before loop

link_pop = f'https://api.worldbank.org/v2/country/DZA/indicator/NY.GDP.PCAP.KD?date=2015:2024&format=json'
response_pop = requests.get(link_pop).json()

# pprint(response_pop)


pop_list = []

pop_list.append({
    'iso3Code' : response_pop[1][0]['countryiso3code'],
    'iso2Code' : response_pop[1][0]['country']['id'],
    'country' : response_pop[1][0]['country']['value'],              
            'pop_2023' : response_pop[1][0]['value'] ,
            'pop_2022' :response_pop[1][1]['value'] ,
            'pop_2021' :response_pop[1][2]['value'] ,
            'pop_2020' : response_pop[1][3]['value'],
            'pop_2019' :response_pop[1][4]['value'] ,
            'pop_2018' :response_pop[1][5]['value'] ,
            'pop_2017' : response_pop[1][6]['value'],
            'pop_2016' : response_pop[1][7]['value'],
            'pop_2015' : response_pop[1][8]['value']
    })

pop_list

[{'iso3Code': 'DZA',
  'iso2Code': 'DZ',
  'country': 'Algeria',
  'pop_2023': 4660.40545686886,
  'pop_2022': 4544.46688059647,
  'pop_2021': 4456.74687641147,
  'pop_2020': 4363.68533766912,
  'pop_2019': 4672.66408682198,
  'pop_2018': 4717.00358853556,
  'pop_2017': 4742.90075536604,
  'pop_2016': 4768.73140080218,
  'pop_2015': 4685.05902729002}]

In [33]:
# this code will loop through all the listed countries and the needed indicators to generate 1 lists for each indicator

print(f"start code")
print('---------------------------')

# indicator = ['NY.GDP.PCAP.KD','SP.POP.TOT']

gdpcapita_list = []
pop_list = []

# each country has its onwn end Point, so need to loop through all end points and extract needed data

for i in country_list :

    print(f"--------------------------")
    print(f"country {i}")

    ## collect data for 1st indicator : Population > SP.POP.TOTL

    try:

        link_pop = f'https://api.worldbank.org/v2/country/{i}/indicator/SP.POP.TOTL?date=2015:2024&format=json'
        response_pop = requests.get(link_pop).json()

        # pprint(response_pop)

        pop_list.append({
            'iso3Code' : response_pop[1][0]['countryiso3code'],
            'iso2Code' : response_pop[1][0]['country']['id'],
            'country' : response_pop[1][0]['country']['value'],              
            'pop_2023' : response_pop[1][0]['value'] ,
            'pop_2022' :response_pop[1][1]['value'] ,
            'pop_2021' :response_pop[1][2]['value'] ,
            'pop_2020' : response_pop[1][3]['value'],
            'pop_2019' :response_pop[1][4]['value'] ,
            'pop_2018' :response_pop[1][5]['value'] ,
            'pop_2017' : response_pop[1][6]['value'],
            'pop_2016' : response_pop[1][7]['value'],
            'pop_2015' : response_pop[1][8]['value']
            })
    except:   
        print("Country not found POP. Skipping...")


    ## collect data for 2nd indicator : GDP per capita > NY.GDP.PCAP.KD


    try:  
        link_gdp = f'https://api.worldbank.org/v2/country/{i}/indicator/NY.GDP.PCAP.KD?date=2015:2024&format=json'
        response_gdp = requests.get(link_gdp).json()

        # pprint(response_gdp)

        gdpcapita_list.append({
            'iso3Code' : response_gdp[1][0]['countryiso3code'],
            'iso2Code' : response_gdp[1][0]['country']['id'],
            'country' : response_gdp[1][0]['country']['value'],
            'GDPcap_2023' : response_gdp[1][0]['value'] ,
            'GDPcap_2022' : response_gdp[1][1]['value'] ,
            'GDPcap_2021' : response_gdp[1][2]['value'] ,
            'GDPcap_2020' : response_gdp[1][3]['value'],
            'GDPcap_2019' :response_gdp[1][4]['value'] ,
            'GDPcap_2018' :response_gdp[1][5]['value'] ,
            'GDPcap_2017' : response_gdp[1][6]['value'],
            'GDPcap_2016' : response_gdp[1][7]['value'],
            'GDPcap_2015' : response_gdp[1][8]['value']
            })
        

    except: 
        print("Country not found GDP. Skipping...")



start code
---------------------------
--------------------------
country AFG
--------------------------
country ALB
--------------------------
country DZA
--------------------------
country ATG
--------------------------
country ARG
--------------------------
country ARM
--------------------------
country AUS
--------------------------
country AUT
--------------------------
country AZE
--------------------------
country BHR
--------------------------
country BGD
--------------------------
country BRB
--------------------------
country BLR
--------------------------
country BEL
--------------------------
country BLZ
--------------------------
country BEN
--------------------------
country BTN
--------------------------
country BOL
--------------------------
country BWA
--------------------------
country BRA
--------------------------
country BGR
--------------------------
country BDI
--------------------------
country KHM
--------------------------
country CMR
-------------------------

In [34]:
## Create DF 1st indicator

pop_df = pd.DataFrame(pop_list)
pop_df = pop_df.dropna(how='all')

pop_df.head()

,iso3Code,iso2Code,country,pop_2023,pop_2022,pop_2021,pop_2020,pop_2019,pop_2018,pop_2017,pop_2016,pop_2015
0,AFG,AF,Afghanistan,41454761,40578842,40000412,39068979,37856121,36743039,35688935,34700612,33831764
1,ALB,AL,Albania,2745972,2777689,2811666,2837849,2854191,2866376,2873457,2876101,2880703
2,DZA,DZ,Algeria,46164219,45477389,44761099,44042091,43294546,42505035,41689299,40850721,40019529
3,ATG,AG,Antigua and Barbuda,93316,92840,92349,91846,91364,90926,90468,89969,89409
4,ARG,AR,Argentina,45538401,45407904,45312281,45191965,44973465,44654882,44288894,43900313,43477012


In [35]:
## Create DF 2nd indicator

gdp_df = pd.DataFrame(gdpcapita_list)

gdp_df = gdp_df.dropna(how='all')

gdp_df.head()

,iso3Code,iso2Code,country,GDPcap_2023,GDPcap_2022,GDPcap_2021,GDPcap_2020,GDPcap_2019,GDPcap_2018,GDPcap_2017,GDPcap_2016,GDPcap_2015
0,AFG,AF,Afghanistan,379.707497,377.665627,408.625855,527.834554,557.861533,553.125152,562.769574,563.872337,565.569730
1,ALB,AL,Albania,5419.637791,5154.827498,4858.052015,4417.041051,4542.268663,4431.555595,4249.820049,4090.372728,3952.803574
2,DZA,DZ,Algeria,4660.405457,4544.466881,4456.746876,4363.685338,4672.664087,4717.003589,4742.900755,4768.731401,4685.059027
3,ATG,AG,Antigua and Barbuda,18090.773518,17507.389224,16071.084322,14935.615024,18508.978373,18038.853295,16951.506354,16635.731889,16080.658050
4,ARG,AR,Argentina,12933.249734,13182.793395,12549.281170,11393.050596,12706.397811,13058.328545,13520.112985,13265.886064,13679.626498


In [36]:
# merge both dataframes in one using iso3code as common key

macro_data_df = pd.merge(pop_df, gdp_df, on='iso3Code')

# drop duplicated columns
macro_data_df = macro_data_df.drop(columns={'iso2Code_y','country_y'})

# rename columns 
macro_data_df = macro_data_df.rename(columns={'iso2Code_x': 'iso2Code',
                                              'country_x': 'country'
                                              })
macro_data_df

,iso3Code,iso2Code,country,pop_2023,pop_2022,pop_2021,pop_2020,pop_2019,pop_2018,pop_2017,...,pop_2015,GDPcap_2023,GDPcap_2022,GDPcap_2021,GDPcap_2020,GDPcap_2019,GDPcap_2018,GDPcap_2017,GDPcap_2016,GDPcap_2015
0,AFG,AF,Afghanistan,41454761,40578842,40000412,39068979,37856121,36743039,35688935,...,33831764,379.707497,377.665627,408.625855,527.834554,557.861533,553.125152,562.769574,563.872337,565.569730
1,ALB,AL,Albania,2745972,2777689,2811666,2837849,2854191,2866376,2873457,...,2880703,5419.637791,5154.827498,4858.052015,4417.041051,4542.268663,4431.555595,4249.820049,4090.372728,3952.803574
2,DZA,DZ,Algeria,46164219,45477389,44761099,44042091,43294546,42505035,41689299,...,40019529,4660.405457,4544.466881,4456.746876,4363.685338,4672.664087,4717.003589,4742.900755,4768.731401,4685.059027
3,ATG,AG,Antigua and Barbuda,93316,92840,92349,91846,91364,90926,90468,...,89409,18090.773518,17507.389224,16071.084322,14935.615024,18508.978373,18038.853295,16951.506354,16635.731889,16080.658050
4,ARG,AR,Argentina,45538401,45407904,45312281,45191965,44973465,44654882,44288894,...,43477012,12933.249734,13182.793395,12549.281170,11393.050596,12706.397811,13058.328545,13520.112985,13265.886064,13679.626498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,PRI,PR,Puerto Rico,3205691,3220113,3262693,3281557,3193694,3193354,3325286,...,3473232,30122.841225,29819.888444,28416.779898,28145.005347,30181.149849,29686.968855,29809.234989,29961.543010,29763.488301
163,PYF,PF,French Polynesia,281118,280378,279792,279209,279034,279815,280087,...,277866,NaN,21042.071447,20184.679397,19811.810232,21328.076136,20709.406300,20317.080196,19528.938471,19166.959476
164,NRU,NR,Nauru,11875,11801,11709,11643,11587,11477,11324,...,10954,8463.132452,8466.192067,8282.366839,7769.406946,7655.243403,7124.017604,7309.381386,7898.227701,7703.431499
165,STP,ST,Sao Tome and Principe,230871,226305,221961,217435,213392,210187,206970,...,200291,1380.601694,1403.233567,1428.286761,1430.841193,1420.660083,1413.843486,1375.594033,1342.533808,1298.109466


In [39]:
# Unpivot the DataFrame to have long data

macro_data_long_df = pd.melt(macro_data_df, 
                  id_vars=["iso3Code",'iso2Code', 'country'], 
                  var_name="ind_y", 
                  value_name="value")
print("\nLong Format:")
print(macro_data_long_df)


Long Format:
     iso3Code iso2Code                country        ind_y         value
0         AFG       AF            Afghanistan     pop_2023  4.145476e+07
1         ALB       AL                Albania     pop_2023  2.745972e+06
2         DZA       DZ                Algeria     pop_2023  4.616422e+07
3         ATG       AG    Antigua and Barbuda     pop_2023  9.331600e+04
4         ARG       AR              Argentina     pop_2023  4.553840e+07
...       ...      ...                    ...          ...           ...
3001      PRI       PR            Puerto Rico  GDPcap_2015  2.976349e+04
3002      PYF       PF       French Polynesia  GDPcap_2015  1.916696e+04
3003      NRU       NR                  Nauru  GDPcap_2015  7.703431e+03
3004      STP       ST  Sao Tome and Principe  GDPcap_2015  1.298109e+03
3005      VUT       VU                Vanuatu  GDPcap_2015  2.746592e+03

[3006 rows x 5 columns]


In [40]:
# split ind_y column to get the indicator + the year

macro_data_long_df[['indicator', 'year']] = macro_data_long_df['ind_y']\
                                                        .str.split('_', expand=True)

#extract only needed columns 

macro_data_long_df = macro_data_long_df[['iso3Code', 'iso2Code', 'country', 'indicator','value', 'year']]

macro_data_long_df.head()

,iso3Code,iso2Code,country,indicator,value,year
0,AFG,AF,Afghanistan,pop,41454761.0,2023
1,ALB,AL,Albania,pop,2745972.0,2023
2,DZA,DZ,Algeria,pop,46164219.0,2023
3,ATG,AG,Antigua and Barbuda,pop,93316.0,2023
4,ARG,AR,Argentina,pop,45538401.0,2023


In [ ]:
# macro_long_df = macro_long_df[['iso3Code', 'iso2Code', 'country', 'indicator','value', 'year']]
# macro_long_df.head()

In [41]:
# change datatype

macro_long_df = macro_data_long_df.astype({
                                        'year' : 'int64',
                                        'value': 'float'})
macro_long_df.dtypes

iso3Code      object
iso2Code      object
country       object
indicator     object
value        float64
year           int64
dtype: object

In [42]:
# export to csv

macro_data_long_df.to_csv('Output/macro_data.csv')